In [1]:
#Load Libraries
import pandas as pd
import numpy as np
import sqlite3 as sqlite3
import numpy as np
import pickle
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import rbf_kernel
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler
from sklearn.metrics import accuracy_score
from sklearn import svm
from sklearn.model_selection import train_test_split, GridSearchCV, KFold
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.multiclass import OneVsRestClassifier

In [2]:
%%time
train_df=pd.read_csv("data/SMOTE_training_data.csv")
test_df=pd.read_csv("data/testing_data_v2.csv")

Wall time: 4.75 s


In [3]:
xTrain=train_df.loc[:,train_df.columns != 'STAT_CAUSE_DESCR']
yTrain=train_df['STAT_CAUSE_DESCR']
xTrain = xTrain.drop('OWNER_CODE', axis=1)
xTrain=pd.get_dummies(xTrain, columns=['NWCG_REPORTING_AGENCY', 'SOURCE_SYSTEM_TYPE'])
xTest=test_df.loc[:,test_df.columns != 'STAT_CAUSE_DESCR']
xTest = xTest.drop('OWNER_CODE', axis=1)
xTest=pd.get_dummies(xTest, columns=['NWCG_REPORTING_AGENCY', 'SOURCE_SYSTEM_TYPE'])
xTest['NWCG_REPORTING_AGENCY_BOR']=0
xTest=xTest[list(xTrain.columns)]
yTest=test_df['STAT_CAUSE_DESCR']

xTrain.shape
# yTrain.shape

(4241600, 19)

In [4]:
xTest.columns

Index(['CONT_DUR', 'LONGITUDE', 'LATITUDE', 'DISCOVERY_DATE', 'DISCOVERY_DOY',
       'FIRE_SIZE', 'NWCG_REPORTING_AGENCY_BIA', 'NWCG_REPORTING_AGENCY_BLM',
       'NWCG_REPORTING_AGENCY_BOR', 'NWCG_REPORTING_AGENCY_DOD',
       'NWCG_REPORTING_AGENCY_FS', 'NWCG_REPORTING_AGENCY_FWS',
       'NWCG_REPORTING_AGENCY_IA', 'NWCG_REPORTING_AGENCY_NPS',
       'NWCG_REPORTING_AGENCY_ST/C&L', 'NWCG_REPORTING_AGENCY_TRIBE',
       'SOURCE_SYSTEM_TYPE_FED', 'SOURCE_SYSTEM_TYPE_INTERAGCY',
       'SOURCE_SYSTEM_TYPE_NONFED'],
      dtype='object')

In [5]:
xTrain.columns

Index(['CONT_DUR', 'LONGITUDE', 'LATITUDE', 'DISCOVERY_DATE', 'DISCOVERY_DOY',
       'FIRE_SIZE', 'NWCG_REPORTING_AGENCY_BIA', 'NWCG_REPORTING_AGENCY_BLM',
       'NWCG_REPORTING_AGENCY_BOR', 'NWCG_REPORTING_AGENCY_DOD',
       'NWCG_REPORTING_AGENCY_FS', 'NWCG_REPORTING_AGENCY_FWS',
       'NWCG_REPORTING_AGENCY_IA', 'NWCG_REPORTING_AGENCY_NPS',
       'NWCG_REPORTING_AGENCY_ST/C&L', 'NWCG_REPORTING_AGENCY_TRIBE',
       'SOURCE_SYSTEM_TYPE_FED', 'SOURCE_SYSTEM_TYPE_INTERAGCY',
       'SOURCE_SYSTEM_TYPE_NONFED'],
      dtype='object')

In [6]:
counts = yTrain.value_counts()
counts

Arson             385600
Smoking           385600
Equipment Use     385600
Campfire          385600
Structure         385600
Children          385600
Debris Burning    385600
Lightning         385600
Railroad          385600
Powerline         385600
Fireworks         385600
Name: STAT_CAUSE_DESCR, dtype: int64

In [7]:
#Global variable necessary for function
counts = yTrain.value_counts()

#Change count accordingly for your model
def count_over(colname, count=10000):
    if counts[colname] > count:
        return count
    return counts[colname]

#Perform undersampling
UnderSampleRatio = {
    'Lightning' : count_over('Lightning'), 'Debris Burning' : count_over('Debris Burning'), 'Campfire' : count_over('Campfire'), 
    'Equipment Use' : count_over('Equipment Use'),    
    'Arson' : count_over('Arson'), 'Children' : count_over('Children'), 'Railroad' : count_over('Railroad'),
    'Smoking' : count_over('Smoking'), 'Powerline' : count_over('Powerline'),
    'Fireworks' : count_over('Fireworks'), 'Structure' : count_over('Structure')
}

newSampStrat=RandomUnderSampler(sampling_strategy=UnderSampleRatio,random_state=441) #Goal balance all classes
xTrain,yTrain=newSampStrat.fit_resample(xTrain,yTrain) #perform the balancing newX and newY are balanced X and y

In [8]:
counts = yTrain.value_counts()
counts

Fireworks         10000
Debris Burning    10000
Structure         10000
Smoking           10000
Lightning         10000
Children          10000
Equipment Use     10000
Campfire          10000
Powerline         10000
Arson             10000
Railroad          10000
Name: STAT_CAUSE_DESCR, dtype: int64

In [9]:
xTrain.shape

(110000, 19)

In [10]:
# RBF kernel with covariate scaling
model_rbf = Pipeline(
    steps=[("scaler", StandardScaler()), 
           ("model", svm.SVC(kernel='rbf', C=10000, gamma="scale", decision_function_shape='ovr', cache_size=1600))]
)

In [11]:
%%time
model_rbf.fit(X=xTrain, y=yTrain)

Wall time: 3h 51min 10s


Pipeline(steps=[('scaler', StandardScaler()),
                ('model', SVC(C=10000, cache_size=1600))])

In [12]:
%%time
yPred = model_rbf.predict(xTest) 

Wall time: 25min 40s


In [13]:
accuracy = accuracy_score(yTest, yPred)
print("Test Accuracy: %.2f%%" % (accuracy * 100.0))

Test Accuracy: 37.30%


In [14]:
%%time
filename='SVM_model_ovr_final'
pickle.dump(model_rbf, open(filename, 'wb')) #Saving the model

Wall time: 20.3 ms


In [ ]:
pickle.dump(yPred, open('SVM_predictions', 'wb'))

In [ ]:
pickle.load(open('SVM_predictions', 'rb'))

In [ ]:
yPred